# Imports

In [24]:
import pandas as pd

import sqlite3

# Load data

In [25]:
df_equip = pd.read_csv('data/raw/russia_losses_equipment.csv')
df_equip.head()

,date,day,aircraft,helicopter,tank,APC,field artillery,MRL,military auto,fuel tank,drone,naval ship,anti-aircraft warfare,special equipment,mobile SRBM system,greatest losses direction,vehicles and fuel tanks,cruise missiles
0,2022-02-25,2,10,7,80,516,49,4,100.0,60.0,0,2,0,NaN,NaN,NaN,NaN,NaN
1,2022-02-26,3,27,26,146,706,49,4,130.0,60.0,2,2,0,NaN,NaN,NaN,NaN,NaN
2,2022-02-27,4,27,26,150,706,50,4,130.0,60.0,2,2,0,NaN,NaN,NaN,NaN,NaN
3,2022-02-28,5,29,29,150,816,74,21,291.0,60.0,3,2,5,NaN,NaN,NaN,NaN,NaN
4,2022-03-01,6,29,29,198,846,77,24,305.0,60.0,3,2,7,NaN,NaN,NaN,NaN,NaN


In [26]:
df_pers = pd.read_csv('data/raw/russia_losses_personnel.csv')
df_pers.head()

,date,day,personnel,personnel*,POW
0,2022-02-25,2,2800,about,0.0
1,2022-02-26,3,4300,about,0.0
2,2022-02-27,4,4500,about,0.0
3,2022-02-28,5,5300,about,0.0
4,2022-03-01,6,5710,about,200.0


# Prepare to upload

In [27]:
df_equip.head(3).T

,0,1,2
date,2022-02-25,2022-02-26,2022-02-27
day,2,3,4
aircraft,10,27,27
helicopter,7,26,26
tank,80,146,150
APC,516,706,706
field artillery,49,49,50
MRL,4,4,4
military auto,100.0,130.0,130.0
fuel tank,60.0,60.0,60.0


In [33]:
df_equip.rename(columns=
                {
                    'date': 'observation_date'
                    , 'day': 'day_of_war'
                    , 'field artillery': 'field_artilery'
                    , 'military auto': 'military_auto'
                    , 'naval ship': 'naval_ship'
                    , 'anti-aircraft warfare': 'anti_aircraft_warfare'
                    , 'special equipment': 'special_equipment'
                    , 'mobile SRBM system': 'mobile_SRBM_system'
                    , 'greatest losses direction': 'greatest_losses_direction'
                    , 'vehicles and fuel tanks': 'vehicles_and_fuel_tanks'
                    , 'cruise missiles': 'cruise_missiles'
                    , 'fuel tank': 'fuel_tank'
                }
               , inplace=True)
df_equip.head(3)

,observation_date,day_of_war,aircraft,helicopter,tank,APC,field_artilery,MRL,military_auto,fuel_tank,drone,naval_ship,anti_aircraft_warfare,special_equipment,mobile_SRBM_system,greatest_losses_direction,vehicles_and_fuel_tanks,cruise_missiles
0,2022-02-25,2,10,7,80,516,49,4,100.0,60.0,0,2,0,NaN,NaN,NaN,NaN,NaN
1,2022-02-26,3,27,26,146,706,49,4,130.0,60.0,2,2,0,NaN,NaN,NaN,NaN,NaN
2,2022-02-27,4,27,26,150,706,50,4,130.0,60.0,2,2,0,NaN,NaN,NaN,NaN,NaN


In [34]:
df_pers.head().T

,0,1,2,3,4
observation_date,2022-02-25,2022-02-26,2022-02-27,2022-02-28,2022-03-01
day_of_war,2,3,4,5,6
personnel,2800,4300,4500,5300,5710
personnel_about,about,about,about,about,about
POW,0.0,0.0,0.0,0.0,200.0


In [43]:
df_pers.rename(columns=
                {
                    'date': 'observation_date'
                    , 'day': 'day_of_war'
                    , 'personnel*': 'personnel_about'
                    , 'POW': 'pows'
                }
               , inplace=True)
df_pers.head(3)

,observation_date,day_of_war,personnel,personnel_about,pows
0,2022-02-25,2,2800,about,0.0
1,2022-02-26,3,4300,about,0.0
2,2022-02-27,4,4500,about,0.0


# Upload

In [44]:
DB_NAME = "russian_losses.db"

con = sqlite3.connect(DB_NAME)

sql = con.cursor()

In [37]:
df_equip.to_sql('tb_equipment', con, if_exists='append', index = False)

339

In [45]:
df_pers.to_sql('tb_personnel', con, if_exists='append', index = False)

339

# Checks

In [46]:
sql.execute('''
    SELECT *
    FROM tb_equipment
    LIMIT 5
''')

pd.DataFrame(sql.fetchall())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,2022-02-25,2,10,7,80,516,49,4,100,60,0,2,0,None,None,None,None,None
1,2022-02-26,3,27,26,146,706,49,4,130,60,2,2,0,None,None,None,None,None
2,2022-02-27,4,27,26,150,706,50,4,130,60,2,2,0,None,None,None,None,None
3,2022-02-28,5,29,29,150,816,74,21,291,60,3,2,5,None,None,None,None,None
4,2022-03-01,6,29,29,198,846,77,24,305,60,3,2,7,None,None,None,None,None


In [47]:
sql.execute('''
    SELECT *
    FROM tb_personnel
    LIMIT 5
''')

pd.DataFrame(sql.fetchall())

,0,1,2,3,4
0,2022-02-25,2,2800,about,0
1,2022-02-26,3,4300,about,0
2,2022-02-27,4,4500,about,0
3,2022-02-28,5,5300,about,0
4,2022-03-01,6,5710,about,200
